In [4]:
from collections import deque

# -----------------------------
# AC-3 Algorithm
# -----------------------------
def revise(domains, X, Y, constraints):
    revised = False
    to_remove = []

    # for x in list(domains[X]):
    #     revised = False  # flag to check if x has any valid y
    #     for y in domains[Y]:
    #         if constraints(X, Y, x, y):
    #             revised = True
    #             break  # no need to check other y's
    #     if not revised:
    #         to_remove.append(x)


    for x in list(domains[X]):
        if not any(constraints(X, Y, x, y) for y in domains[Y]):
            to_remove.append(x)

    if to_remove:
        for x in to_remove:
            domains[X].remove(x)
        revised = True

    return revised


def AC3(variables, domains, neighbors, constraints):
    queue = deque((X, Y) for X in variables for Y in neighbors[X])

    while queue:
        (X, Y) = queue.popleft()
        if revise(domains, X, Y, constraints):
            if not domains[X]:
                return False
            for Z in neighbors[X]:
                if Z != Y:
                    queue.append((Z, X))
    return True


# -----------------------------
# Backtracking Search
# -----------------------------
def backtrack(assignment, variables, domains, neighbors, constraints):
    if set(assignment.keys()) == set(variables):
        return assignment

    var = select_unassigned_var(assignment, variables, domains)

    for value in order_domain_values(var, domains):
        if is_consistent(var, value, assignment, neighbors, constraints):
            assignment[var] = value
            saved_domains = {v: set(domains[v]) for v in variables}

            # Inference with AC-3
            domains[var] = {value}
            if AC3(variables, domains, neighbors, constraints):
                result = backtrack(assignment, variables, domains, neighbors, constraints)
                if result is not None:
                    return result

            # Restore domains
            domains = {v: set(saved_domains[v]) for v in variables}
            del assignment[var]

    return None


def select_unassigned_var(assignment, variables, domains):
    unassigned = [v for v in variables if v not in assignment]
    return min(unassigned, key=lambda var: len(domains[var]))  # MRV heuristic


def order_domain_values(var, domains):
    return list(domains[var])


def is_consistent(var, value, assignment, neighbors, constraints):
    for other_var, other_value in assignment.items():
        if other_var in neighbors[var]:
            if not constraints(var, other_var, value, other_value):
                return False
    return True


# -----------------------------
# Problem Setup
# -----------------------------
variables = ["A", "B", "C", "D", "E", "F", "G"]

# Domains: each variable can be assigned to a day
days = {"Monday", "Tuesday", "Wednesday"}
domains = {v: set(days) for v in variables}

# Neighbors based on given constraints
neighbors = {
    "A": ["B", "C"],
    "B": ["A", "C", "D", "E"],
    "C": ["A", "B", "E", "F"],
    "D": ["B", "E"],
    "E": ["B", "C", "D", "F", "G"],
    "F": ["C", "E", "G"],
    "G": ["E", "F"]
}

# Constraint: connected variables must not share the same day
def constraints(A, B, a, b):
    return a != b


# -----------------------------
# Run Solver
# -----------------------------
print("Running AC-3 preprocessing...")
AC3(variables, domains, neighbors, constraints)

print("Domains after AC-3:", domains)

solution = backtrack({}, variables, domains, neighbors, constraints)

print("\nSolution:", solution)



Running AC-3 preprocessing...
Domains after AC-3: {'A': {'Wednesday', 'Monday', 'Tuesday'}, 'B': {'Wednesday', 'Monday', 'Tuesday'}, 'C': {'Wednesday', 'Monday', 'Tuesday'}, 'D': {'Wednesday', 'Monday', 'Tuesday'}, 'E': {'Wednesday', 'Monday', 'Tuesday'}, 'F': {'Wednesday', 'Monday', 'Tuesday'}, 'G': {'Wednesday', 'Monday', 'Tuesday'}}

Solution: {'A': 'Wednesday', 'B': 'Monday', 'C': 'Tuesday', 'D': 'Tuesday', 'E': 'Wednesday', 'F': 'Monday', 'G': 'Tuesday'}
